In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [350]:
from copy import deepcopy, copy
from fastai2.text.all import L
from functools import wraps, update_wrapper
import inspect
import numpy as np
from string import ascii_lowercase
import types

# from htools import copy_func, rename_params

In [46]:
def copy_func(func):
    new_func = types.FunctionType(func.__code__, func.__globals__,
                                  func.__name__, func.__defaults__,
                                  func.__closure__)
    defaults = getattr(func, '__kwdefaults__') or {}
    new_func.__kwdefaults__ = defaults.copy()
    return update_wrapper(new_func, func)

In [81]:
def rename_params(func, **old2new):
    new_func = copy_func(func)
    sig = inspect.signature(new_func)
    kw_defaults = func.__kwdefaults__ or {}
    names, params = map(list, zip(*sig.parameters.items()))
    for old, new in old2new.items():
        idx = names.index(old)
        default = kw_defaults.get(old) or params[idx].default
        params[idx] = inspect.Parameter(new, params[idx].kind, default=default)
    new_func.__signature__ = sig.replace(parameters=params)
    return new_func

In [48]:
x = 2
getattr(x, '__hash__')

<method-wrapper '__hash__' of int object at 0x108ac55c0>

```
Somehow assignment:

func.__defaults__ = _defaults

is changing _defaults

Assigning

wrapper.__defaults__ = _defaults

does not change _defaults and DOES change wrapper._defaults, but it doesn't change func.__defaults__ and doesn't affect func behavior
```

In [454]:
def immutify_defaults(func):
    # If `__hash__` is not None, object is immutable already.
    # Python sets __defaults__ and __kwdefaults__ to None when they're empty.
    _defaults = tuple(o if getattr(o, '__hash__') else deepcopy(o)
                      for o in getattr(func, '__defaults__') or ()) or None
    _kwdefaults = {k: v if getattr(v, '__hash__') else deepcopy(v) for k, v 
                   in (getattr(func, '__kwdefaults__') or {}).items()} or None

    @wraps(func)
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        wrapper.__defaults__ = func.__defaults__ = deepcopy(_defaults)
        wrapper.__kwdefaults__ = func.__kwdefaults__ = deepcopy(_kwdefaults)
        return res
    return wrapper

In [455]:
arr = [1, 3, 'abc', [2,4,5]]
a2 = [deepcopy(x) for x in arr]
arr, a2

([1, 3, 'abc', [2, 4, 5]], [1, 3, 'abc', [2, 4, 5]])

In [456]:
arr.append(9)
arr, a2

([1, 3, 'abc', [2, 4, 5], 9], [1, 3, 'abc', [2, 4, 5]])

In [457]:
arr[0] = 3
arr, a2

([3, 3, 'abc', [2, 4, 5], 9], [1, 3, 'abc', [2, 4, 5]])

In [458]:
def foo(c, b=3, a=['a', 'b']):
    print(c, b, a)
    a.append(np.random.choice(list(ascii_lowercase)))
    return a

In [459]:
foo(2)

2 3 ['a', 'b']


['a', 'b', 'm']

In [460]:
foo(4)

4 3 ['a', 'b', 'm']


['a', 'b', 'm', 'm']

In [461]:
foo.__defaults__

(3, ['a', 'b', 'm', 'm'])

In [462]:
foo.__kwdefaults__

In [463]:
@immutify_defaults
def foobar(a, *args, b=[3], c=(4, 5)):
    b.append(1)
    return b

In [464]:
foobar(1, 'c', 'd', 'e')

[3, 1]

In [465]:
foobar(2, 'c', 'd', 'e')

[3, 1]

In [466]:
foobar(1, 'c', 'd', 'e')

[3, 1]

In [467]:
foobar.__kwdefaults__

{'b': [3], 'c': (4, 5)}

In [468]:
foobar.__defaults__

In [469]:
@immutify_defaults
def foo(a, b=3, c=['a', 'b']):
    print('IN FOO', a, b, c)
    c.append(np.random.choice(list(ascii_lowercase)))
    return c

In [470]:
foo.__defaults__

In [471]:
foo(4)

IN FOO 4 3 ['a', 'b']


['a', 'b', 'l']

In [472]:
foo(11)

IN FOO 11 3 ['a', 'b']


['a', 'b', 'a']

In [473]:
foo(5)

IN FOO 5 3 ['a', 'b']


['a', 'b', 't']

In [474]:
foo.__defaults__

(3, ['a', 'b'])

In [475]:
foo.__kwdefaults__

In [476]:
foo.__wrapped__.__defaults__

(3, ['a', 'b'])

In [478]:
@immutify_defaults
def bar(x, y=100, z=[1, 2, 3]):
    print(x, y, z)
    z.append(np.random.choice(list(ascii_lowercase)))
    return z

In [479]:
bar(9)

9 100 [1, 2, 3]


[1, 2, 3, 'o']

In [480]:
bar(12)

12 100 [1, 2, 3]


[1, 2, 3, 'n']

In [481]:
bar.__defaults__

(100, [1, 2, 3])

In [482]:
bar.__wrapped__.__defaults__

(100, [1, 2, 3])

In [484]:
def mask(m, n=1, o=2):
    return m*n*o

In [485]:
mask.__defaults__

(1, 2)

In [486]:
foo.__wrapped__.__defaults__

(3, ['a', 'b'])

In [487]:
def mask(m, n):
    return m*n

In [488]:
mask.__defaults__

In [489]:
mask.__kwdefaults__

In [573]:
def add_diag(a, b=3, *args, x=np.arange(9).reshape(3, 3), **kwargs):
    for i, j in zip(*map(range, x.shape)):
        if i == j:
            x[i, j] += 1
    return a, x

In [574]:
add_diag(2)

(2, array([[1, 1, 2],
        [3, 5, 5],
        [6, 7, 9]]))

In [575]:
add_diag(3)

(3, array([[ 2,  1,  2],
        [ 3,  6,  5],
        [ 6,  7, 10]]))

In [576]:
add_diag(4)

(4, array([[ 3,  1,  2],
        [ 3,  7,  5],
        [ 6,  7, 11]]))

In [577]:
@immutify_defaults
def add_diag(a, b=3, *args, x=np.arange(9).reshape(3, 3), **kwargs):
    for i, j in zip(*map(range, x.shape)):
        if i == j:
            x[i, j] += 1
    return a, x

In [578]:
add_diag(2)

(2, array([[1, 1, 2],
        [3, 5, 5],
        [6, 7, 9]]))

In [579]:
add_diag(3)

(3, array([[1, 1, 2],
        [3, 5, 5],
        [6, 7, 9]]))

In [580]:
add_diag(4)

(4, array([[1, 1, 2],
        [3, 5, 5],
        [6, 7, 9]]))